In [ ]:
#r "nuget: Azure.Identity, 1.7.0"
#r "nuget: Microsoft.Azure.Management.Media, 6.0.0"

In [ ]:
using Azure.Core;
using Azure.Identity;
using Microsoft.Azure.Management.Media;
using Microsoft.Azure.Management.Media.Models;
using Microsoft.Rest;
using System.Collections.ObjectModel;

In [ ]:
var configuration = new 
{
    SubscriptionId = Environment.GetEnvironmentVariable("SubscriptionId"),
    ResourceGroupName = "rg-improove-course-management",
    AccountName = "mediavdavmal",
};

In [ ]:
var authToken = await new DefaultAzureCredential().GetTokenAsync(new TokenRequestContext(new []{ "https://management.core.windows.net/.default"}));
var client = new AzureMediaServicesClient(new Uri("https://management.azure.com/"), (ServiceClientCredentials)new Microsoft.Rest.TokenCredentials(authToken.Token, "Bearer"))
{
    SubscriptionId = configuration.SubscriptionId
};

In [ ]:

var locator = await client.StreamingLocators.CreateAsync(
    resourceGroupName: configuration.ResourceGroupName,
    accountName: configuration.AccountName,
    streamingLocatorName: Guid.NewGuid().ToString("N"),
    new StreamingLocator
    {
        AssetName = Environment.GetEnvironmentVariable("StreamingLocator_AssetName"),
        StreamingPolicyName = PredefinedStreamingPolicy.ClearStreamingOnly
    }
);


In [ ]:
var streamingUriCollection = new Collection<Uri>();

var endpoint = await client.StreamingEndpoints.GetAsync(configuration.ResourceGroupName, configuration.AccountName, "default");
if (endpoint.ResourceState != StreamingEndpointResourceState.Running)
{
    await client.StreamingEndpoints.StartAsync(configuration.ResourceGroupName, configuration.AccountName, "default");
}
var paths = await client.StreamingLocators.ListPathsAsync(configuration.ResourceGroupName, configuration.AccountName, locator.Name);

foreach (var path in paths.StreamingPaths)
{
    var builder = new UriBuilder
    {
        Scheme = "https",
        Host = endpoint.HostName,

        Path = path.Paths[0]
    };
    streamingUriCollection.Add(builder.Uri);
}

In [ ]:
foreach (var uri in streamingUriCollection) 
{
    Console.WriteLine(uri);
}